In [1]:
import sys
from pathlib import Path
from io import BytesIO
import json

import torch as t
import numpy as np
import webdataset as wds

In [11]:
data_path = "../data/wiki_sentences_v1.tar"
dataset = wds.WebDataset(data_path)
dataset_iter = iter(dataset)

In [12]:
class SparseRecoder(t.nn.Module):
    def __init__(self, n_in, n_hidden, n_out, add_topk=True):
        super(SparseRecoder, self).__init__()
        self.encoder = t.nn.Linear(n_in, n_hidden)
        self.decoder = t.nn.Linear(n_hidden, n_out)
        self.add_topk = add_topk

    def forward(self, x):
        encoded = self.encoder(x)
        if self.add_topk:
            # TODO add topk
            pass
        decoded = self.decoder(encoded)
        return decoded

In [13]:
def unpack(sample):
    text = sample["text.txt"]
    resid_gpt2 = np.load(BytesIO(sample["resid_gpt2.npy"]))
    resid_opt = np.load(BytesIO(sample["resid_opt.npy"]))
    return text, resid_gpt2, resid_opt

txt, resid_gpt2, resid_opt = unpack(next(dataset_iter))
d_first = resid_gpt2.shape[-1]
d_second = resid_opt.shape[-1]

In [14]:
from tqdm.notebook import tqdm

In [18]:
expansion_factor = 8
lr = 1e-3
sae_first = SparseRecoder(d_first, expansion_factor * d_first, d_first, add_topk=False)
sae_first = sae_first.cuda()
optimizer = t.optim.Adam(sae_first.parameters(), lr=lr)

# max_steps = 100

for i, sample in tqdm(enumerate(dataset_iter)):
    txt, resid_gpt2, resid_opt = unpack(sample)
    
    resid_gpt2 = t.tensor(resid_gpt2, dtype=t.float32, device="cuda")[0, ...]
    reconstr_gpt2 = sae_first(resid_gpt2)
    
    loss = t.nn.functional.mse_loss(reconstr_gpt2, resid_gpt2)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad(    )
    # if i == 100:
    #     break

0it [00:00, ?it/s]

In [19]:
print(loss)

tensor(0.3392, device='cuda:0', grad_fn=<MseLossBackward0>)
